## Testing String Manipulation and Functions

In [2]:
from datetime import datetime
import string

class split_converter:
    def split_to_lap(splits:list):
        ''' Take an input list of all the race splits and return list of laps '''

        splits = split_converter.fix_empty_splits(splits)
        conversion = [split_converter.interpret_input(y) - split_converter.interpret_input(x) for x,y in zip(splits,splits[1:])]
        formatted_conversion = [str(x) for x in conversion]

        return formatted_conversion

    def interpret_input(input:str):
        ''' Convert string to useable datetime value '''
        time_val = datetime.strptime(input, '%M:%S.%f')
        return time_val
    
    def fix_empty_splits(raw_input:list):
        if not any(bool(i) for i in raw_input):
            return raw_input
        
        

test = ('1:23.45', '2:40.43', '3:50.81', '3:59.49') + ('3:50.46', '3:50.44', '3:50.41', '3:50.43')

test2 = split_converter.interpret_input('1:23.45')
test3 = split_converter.interpret_input('2:34.56')
test4 = ['1:07.95', '0:52.90']

test5 = ['1:25.00', '1:34.81', '1:38.13', '1:37.47']

class time_addition:

    def add_time(str_list:list or tuple):
        minutes = sum([int(time[0]) for time in str_list])
        seconds = sum([float(sec[2:]) for sec in str_list])

        minutes += int(seconds // 60)
        remaining_seconds = round(seconds % 60, 2)

        return f'{minutes}:{remaining_seconds:02}'
    
# [i for i in test5 if i != ''] == [i for i in test5 if i]
from time_math import time_addition as ta
ta.add_time(test5)


'6:15.41'

## Replacing Tables in the Database

In [1]:
from backup_database import db_backupper
import sqlite3

db_name = db_backupper().db_name

counter = 1

running_db = sqlite3.connect(db_name)
cursor = running_db.cursor()

for _ in range(2):
    if counter % 2 != 0:
        cursor.execute('''DROP TABLE relays''')
        print('dropped the table')
        print(counter)
        counter += 1
    else:
        cursor.execute('''
                CREATE TABLE IF NOT EXISTS relays (
                        meet_name TEXT,
                        date DATE,
                        event TEXT,
                        sex TEXT,
                        leg_1 TEXT,
                        leg_2 TEXT,
                        leg_3 TEXT,
                        leg_4 TEXT,
                        time TIME,
                        PRIMARY KEY (meet_name, date, event, sex, time)
                    )
                   ''')
        print('added the table')
        print(counter)
        counter += 1

running_db.commit()
running_db.close()


dropped the table
1
added the table
2


## Troubleshoot time_math

In [1]:
from time_math import time_addition

# time_addition.add_time(test4)

data1 = ['0:36.72',	'0:43.94', '0:43.78', '0:44.00']
time_addition.add_time(data1)


'2:48.44'

## Troubleshoot split_conversions

In [15]:
from split_conversions import split_converter

test6 = ['1:02.95', '2:52.87', '', '', '', '', '', '']
test7 = ['2:05.12', '4:05.12', '6:02.13', '8:06.23', '10:12.85', '12:34.56', '14:56.42', '17:00.29']


temp1 = [split_converter.interpret_input(y) - split_converter.interpret_input(x) for x,y in zip(test7,test7[1:])]
[str(i)[3:-4] if len(str(i)) > 7 else str(i)[3:] + '.00' for i in temp1] 


['2:00.00', '1:57.01', '2:04.10', '2:06.62', '2:21.71', '2:21.86', '2:03.87']

## Meet name db query

In [8]:
from backup_database import db_backupper
from database_operations import db_ops

db_name = db_backupper.db_name
db_ops.retrieve_meets(db_name)

['ahsaa 6a section 3 sectional meet',
 'ahsaa 6a state meet',
 'homewood invitational',
 'jaguar invitational',
 'king of the mountain invitational',
 'king of the west invitational',
 'mountain brook invitational',
 'northridge black and blue invitational',
 'paul w. bryant high school invitational',
 'southern mississippi high school invitational',
 'spain park invitational',
 'tuscaloosa/western alabama city/county championships']

### Troubleshoot new create_database classes

In [12]:
import sqlite3  
import os
# https://www.w3schools.com/sql/sql_view.asp


class db_creator:
    def __init__(self, db_name = 'test.db'):
        self.db_name = db_name

    def create_db(self):
        db_name = self.db_name
        running_db = sqlite3.connect(db_name)
        connection = running_db.cursor()

        connection.execute('''
                    CREATE TABLE IF NOT EXISTS athletes (
                        athlete TEXT,
                        season TEXT,
                        season_year YEAR,
                        grade INTEGER,
                        sex TEXT,
                        PRIMARY KEY (athlete, season, season_year)
                    )
                    ''')

        connection.execute('''
                    CREATE TABLE IF NOT EXISTS db800 (
                        athlete TEXT,
                        first_200 DOUBLE,
                        second_200 DOUBLE,
                        third_200 DOUBLE,
                        fourth_200 DOUBLE,
                        first_400 DOUBLE,
                        second_400 DOUBLE,
                        time_200s AS (first_200 + second_200 + third_200 + fourth_200),
                        time_400s AS (first_400 + second_400),
                        PRIMARY KEY (athlete, first_200, second_200, third_200, fourth_200, first_400, second_400)
                        )
                    ''')

        connection.execute('''
                    CREATE TABLE IF NOT EXISTS db400 (
                        athlete TEXT,
                        first_200 DOUBLE,
                        second_200 DOUBLE,
                        time_sec AS (first_200 + second_200),
                        PRIMARY KEY (athlete, first_200, second_200)
                        )
                    ''')


        connection.execute('''
                        CREATE TABLE IF NOT EXISTS race_laps (
                            athlete TEXT,
                            event_distance_m INTEGER,
                            date DATE,
                            meet_name TEXT,
                            relay_y_n INTEGER,
                            lap_1 TEXT,
                            lap_2 TEXT,
                            lap_3 TEXT,
                            lap_4 TEXT,
                            lap_5 TEXT,
                            lap_6 TEXT,
                            lap_7 TEXT,
                            lap_8 TEXT,
                            total_time TEXT,
                            PRIMARY KEY (athlete, event_distance_m, date, meet_name, total_time)
                        )
                    ''')

        connection.execute('''
                        CREATE TABLE IF NOT EXISTS meet_information (
                            name TEXT,
                            date DATE,
                            host_school TEXT,
                            physical_location TEXT,
                            city_state TEXT,
                            weather_temperature_deg_F INTEGER,
                            weather_clouds TEXT,
                            weather_precipitation TEXT,
                            weather_notes TEXT,
                            PRIMARY KEY(name, date)
                    )
                    ''')

        connection.execute('''
                        CREATE TABLE IF NOT EXISTS relays (
                            meet_name TEXT,
                            date DATE,
                            event TEXT,
                            sex TEXT,
                            leg_1 TEXT,
                            leg_2 TEXT,
                            leg_3 TEXT,
                            leg_4 TEXT,
                            time TIME,
                            PRIMARY KEY (meet_name, date, event)
                        )
                        ''')
        try:
            # Check if a database of the same name exists
            if self.db_name not in os.listdir(path = os.getcwd()):
                running_db.commit()
                running_db.close()
            else: 
                raise Exception

        except:
            overwrite_permission = input('Overwrite most recent database? [y]/n ')
            if overwrite_permission in ['', 'y']:
                running_db.commit()
                print('Database overwritten.')
            else:
                print('Commit canceled. Rename database.')
        
test1 = db_creator('test.db')
test1.create_db()



Commit canceled. Rename database.
